In [1]:
import sys
import os
sys.path.insert(1, '/storage/home/nxt5197/work/530_stellar_atmospheres/lib/')
from statmospheres import Partition
# from statmospheres import Phi
from statmospheres import NIST_Phi
from statmospheres import A_sol
# from statmospheres import Solve_Pe
from statmospheres import NIST_Solve_Pe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('../lib/plt_format.mplstyle')
from astropy import units as u
from astropy import constants as const

### Equation (9.8) Gray
$P_e = P_g \frac{\sum A_j \frac{\Phi_j(T)/P_e}{1 + \Phi_j(T)/P_e}}{\sum A_j \Big[ 1 + \frac{\Phi_j/P_e}{1 + \Phi_j(T)/P_e} \Big]}$

In [2]:
os.chdir('/storage/home/nxt5197/work/530_stellar_atmospheres/lib')

In [3]:
Partition_table=pd.read_csv('RepairedPartitionFunctions.txt',delimiter=r"\s+", header=None)
PTable=Partition_table.rename(columns={0: 'Element', 1: 0.2,2: 0.4,3: 0.6,4: 0.8,5: 1.0,6: 1.2,7: 1.4,8: 1.6,9: 1.8,10: 2.0,11: 'log g0'})
PTable


,Element,0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8,2.0,log g0
0,H,0.368,0.303,0.301,0.301,0.301,0.301,0.301,0.301,0.301,0.301,0.301
1,He,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,He+,0.301,0.301,0.301,0.301,0.301,0.301,0.301,0.301,0.301,0.301,0.301
3,Li,-,0.987,0.488,0.359,0.320,0.308,0.304,0.302,0.302,0.302,0.301
4,Be,-,0.328,0.087,0.025,0.007,0.002,0.001,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
242,Pa+,2.985,2.413,2.169,1.990,1.846,1.723,1.615,1.515,1.419,1.323,1.519
243,Pa++,-,2.106,1.845,1.652,1.469,1.281,1.088,0.895,0.708,0.532,-
244,U,3.221,2.696,2.298,2.032,1.849,1.719,1.623,1.552,1.498,1.455,1.929
245,U+,2.694,2.031,1.719,1.553,1.455,1.392,1.349,1.317,1.292,1.270,1.716


In [4]:
T = 5776*u.K
element = 'Li'

Partition(PTable,element,T)

2.2123184714236346

In [5]:
NistIonizTable = pd.read_csv('nist_ioniz.txt',delimiter='\t',header=None)
NistIonizTable=NistIonizTable.rename(columns={0: 'Atomic_Number', 1: 'Element',2: 'Atomic_Weight',3: 'Ionization_Energy'})
NistIonizTable['Element'] = NistIonizTable['Element'].str.strip()
# IonizTable = pd.read_fwf('ioniz.txt',header=None)
# IonizTable=IonizTable.rename(columns={0: 'Atomic_Number', 1: 'Element',2: 'Atomic_Weight',3: 'Ionization_Energy_1',4: 'Ionization_Energy_2',5: 'Ionization_Energy_3'})
NistIonizTable[NistIonizTable['Element']=='H']

,Atomic_Number,Element,Atomic_Weight,Ionization_Energy
0,1,H,1.008,13.598434


In [6]:
df=NistIonizTable
element='H'
row = df[df.Element==element]
X = row['Ionization_Energy'].item()*u.eV
X

<Quantity 13.59843401 eV>

In [7]:
element='Ba'
T = 7715*u.K
NIST_Phi(NistIonizTable,PTable,element,T)

<Quantity 833555.13949913>

In [8]:
ATable = pd.read_csv('SolarAbundance.txt',delimiter='\t',header=0)
df=ATable
element='Li'
df[df.element==element]

def Aj(j):
    return A_sol(ATable,j).A.item()

Aj('Li')

1.26e-11

In [9]:
# Partition(PTable,'Li',T)
df = PTable
element='Tm'
row = df[df.Element==element]
list(row.reset_index(drop=True).iloc[0][1:-1])

['-', '-', '-', 1.192, 1.038, 0.964, 0.93, 0.916, 0.911, 0.909]

In [10]:
element='Eu'
theta = 5040*u.K/T
row = df[df.Element==element]
thetas = list(df)[1:-1]
U_strs = [val for val in list(row.reset_index(drop=True).iloc[0][1:-1])]
nulls = []
for i,U in enumerate(U_strs):
    if U == '-':
        nulls.append(i)
for i in reversed(nulls):
    del thetas[i]
    del U_strs[i]
Us = [float(u) for u in U_strs]
U_r = np.interp(theta,thetas,Us)
U_r

1.396352559948153

In [11]:
# list(PTable.Element[1:])

['He',
 'He+',
 'Li',
 'Be',
 'Be+',
 'B',
 'B+',
 'C',
 'C+',
 'C++',
 'N',
 'N+',
 'O',
 'O+',
 'F',
 'F+',
 'Ne',
 'Ne+',
 'Na',
 'Na+',
 'Mg',
 'Mg+',
 'Mg++',
 'Al',
 'Al+',
 'Al++',
 'Si',
 'Si+',
 'Si++',
 'P',
 'P+',
 'P++',
 'S',
 'S+',
 'Cl',
 'Cl+',
 'Ar',
 'Ar+',
 'K',
 'K+',
 'Ca',
 'Ca+',
 'Ca++',
 'Sc',
 'Sc+',
 'Sc++',
 'Ti',
 'Ti+',
 'Ti++',
 'V',
 'V+',
 'V++',
 'Cr',
 'Cr+',
 'Cr++',
 'Mn',
 'Mn+',
 'Fe',
 'Fe+',
 'Fe++',
 'Co',
 'Co+',
 'Co++',
 'Ni',
 'Ni+',
 'Ni++',
 'Cu',
 'Cu+',
 'Cu++',
 'Zn',
 'Zn+',
 'Zn++',
 'Ga',
 'Ga+',
 'Ga++',
 'Ge',
 'Ge+',
 'Ge++',
 'As',
 'As+',
 'As++',
 'Se',
 'Se+',
 'Br',
 'Br+',
 'Kr',
 'Kr+',
 'Rb',
 'Rb+',
 'Sr',
 'Sr+',
 'Sr++',
 'Y',
 'Y+',
 'Y++',
 'Zr',
 'Zr+',
 'Zr++',
 'Nb',
 'Nb+',
 'Nb++',
 'Mo',
 'Mo+',
 'Mo++',
 'Tc',
 'Tc+',
 'Tc++',
 'Ru',
 'Ru+',
 'Ru++',
 'Rh',
 'Rh+',
 'Rh++',
 'Pd',
 'Pd+',
 'Pd++',
 'Ag',
 'Ag+',
 'Cd',
 'Cd+',
 'Cd++',
 'In',
 'In+',
 'In++',
 'Sn',
 'Sn+',
 'Sn++',
 'Sb',
 'Sb+',
 'Sb++',
 'T

In [12]:
T = 5733*u.K
elements=list(ATable[ATable.A>0].element)
for element in elements:
    if element not in list(PTable.Element):
        elements.remove(element)
    if element not in list(NistIonizTable.Element):
        elements.remove(element)
    if element!= 'H' and element+'+' not in list(PTable.Element[1:]):
        elements.remove(element)
# elements.insert(0, 'H')
print(elements)
Pg = 10**4.95
Pef = NIST_Solve_Pe(Pg,elements,ATable,NistIonizTable,PTable,T)
print(np.log10(Pef))

['H', 'He', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'W', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Th', 'U']
It took 46 iterations to calculate P_e
1.1493493567616038


In [13]:
np.log10(np.sqrt(Pg*NIST_Phi(NistIonizTable,PTable,'H',T)))

<Quantity 0.95704387>

In [15]:
Ts = [4310,4325,4345,4370,4405,4445,4488,4524,4561,4608,4660,4720,4800,4878,4995,5132,5294,5490,5733,6043,6429,6904,7467,7962,8358,8630,8811]
Pgs = [2.87,3.03,3.17,3.29,3.41,3.52,3.64,3.75,3.86,3.97,4.08,4.19,4.30,4.41,4.52,4.63,4.74,4.85,4.95,5.03,5.10,5.15,5.18,5.21,5.23,5.26,5.29]
Pes = [-1.16,-1.02,-0.89,-0.78,-0.66,-0.55,-0.44,-0.33,-0.23,-0.12,-0.01,0.10,0.22,0.34,0.47,0.61,0.76,0.93,1.15,1.43,1.78,2.18,2.59,2.92,3.16,3.32,3.42]
data = {'Temp': Ts, 'log_Pg': Pgs, 'log_Pe': Pes}
Table_9o2 = pd.DataFrame(data)
# Table_9o2

In [16]:
talbelen=len(Ts)
calc_Pe = [np.log10(NIST_Solve_Pe(10**Table_9o2.log_Pg.iloc[i],elements,ATable,NistIonizTable,PTable,Table_9o2.Temp.iloc[i]*u.K)) for i in range(talbelen)]

It took 17 iterations to calculate P_e
It took 18 iterations to calculate P_e
It took 20 iterations to calculate P_e
It took 20 iterations to calculate P_e
It took 21 iterations to calculate P_e
It took 21 iterations to calculate P_e
It took 22 iterations to calculate P_e
It took 22 iterations to calculate P_e
It took 23 iterations to calculate P_e
It took 23 iterations to calculate P_e
It took 24 iterations to calculate P_e
It took 24 iterations to calculate P_e
It took 24 iterations to calculate P_e
It took 24 iterations to calculate P_e
It took 24 iterations to calculate P_e
Count exceeded. While loop broken.
Pef: 3.9812581884688854
Pei: 3.9812581884688836
It took 100 iterations to calculate P_e
It took 27 iterations to calculate P_e
It took 33 iterations to calculate P_e
It took 46 iterations to calculate P_e
Count exceeded. While loop broken.
Pef: 26.879248315575577
Pei: 26.879248315575563
It took 100 iterations to calculate P_e
Count exceeded. While loop broken.
Pef: 60.260665745

In [17]:
Table_9o2['calc_Pe'] = [round(x.value,2) for x in calc_Pe]
Table_9o2['Pe_diff'] = [Table_9o2.calc_Pe.iloc[i] - Table_9o2.log_Pe.iloc[i] for i in range(talbelen)]
Table_9o2

,Temp,log_Pg,log_Pe,calc_Pe,Pe_diff
0,4310,2.87,-1.16,-1.18,-0.02
1,4325,3.03,-1.02,-1.03,-0.01
2,4345,3.17,-0.89,-0.90,-0.01
3,4370,3.29,-0.78,-0.79,-0.01
4,4405,3.41,-0.66,-0.68,-0.02
5,4445,3.52,-0.55,-0.57,-0.02
6,4488,3.64,-0.44,-0.45,-0.01
7,4524,3.75,-0.33,-0.35,-0.02
8,4561,3.86,-0.23,-0.24,-0.01
9,4608,3.97,-0.12,-0.13,-0.01


In [18]:
SumA = 0
SumAmu = 0
for i in range(30):
    SumA += ATable.A.iloc[i].item()
    SumAmu += ATable.weight.iloc[i].item()*ATable.A.iloc[i].item()
print(f'Sum of Abundances: {SumA}\nSum of Abundances times weights: {SumAmu}')

Sum of Abundances: 1.0861752053727
Sum of Abundances times weights: 1.367107621952563


In [19]:
SumA = 0
SumAmu = 0
for i in range(2,len(ATable)):
    if ATable.A.iloc[i].item() > 0:
        SumA += ATable.A.iloc[i].item()
        SumAmu += ATable.weight.iloc[i].item()*ATable.A.iloc[i].item()
print(f'Sum of Abundances: {SumA}\nSum of Abundances times weights: {SumAmu}')

Sum of Abundances: 0.0010752126721989992
Sum of Abundances times weights: 0.018453079261262224
